# Subplotter

This code automatically makes subplots of a galaxy's masked Stellar velocity, H-alpha velocity, and Flux.

In [1]:
#importing

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import fits
import numpy.ma as ma

In [2]:
data_folder = "/Users/Emlilio/School/Internships/UR ASTRO/Data/" #create a variable for directory to common folder

In [3]:
cube_df = data_folder + 'MaNGA/manga-8997-9102-MAPS-HYB10-MILESHC-MASTARSSP.fits.gz'

In [4]:
drpall = data_folder + 'drpall_ttype.fits'
drpalltt = Table.read(drpall, format="fits",hdu=1)   #create and visualize table for quick reference


In [5]:
#create a dictionary for plateifu#
drpalltt_dict = {}
for i in range(len(drpalltt)):
    plateifu = drpalltt['plateifu'][i]
    drpalltt_dict[plateifu] = i

In [ ]:
max_lim = 400              #creates default plot limit

for i in range(9080,9180):
    plateifu = drpalltt['plateifu'][i]      #calls the specific row by index
    try:
        cube_df = data_folder + 'MaNGA/manga-'+ plateifu +'-MAPS-HYB10-MILESHC-MASTARSSP.fits.gz'
        cube = fits.open(cube_df)                        #file requires opening
        stellar_vel = cube['STELLAR_VEL'].data           #for the following we create variable representing data from file HDU's
        stellar_vel_ivar = cube['STELLAR_VEL_IVAR'].data 
        stellar_mask = cube['STELLAR_VEL_MASK'].data
        halpha_vel = cube['EMLINE_GVEL'].data[23]        #the following three have several channels, but were only concerned with the Halpha spectra (channel 23)
        halpha_gvel_ivar = cube['EMLINE_GVEL_IVAR'].data[23]
        halpha_gvel_mask = cube['EMLINE_GVEL_MASK'].data[23]
        flux = cube['SPX_MFLUX'].data
    
        cube.close()
#creates figure for subplots,  
        fig,axes = plt.subplots(1,3, figsize=(18,6), tight_layout=True)
        fig.suptitle(plateifu)

        mhalpha_vel = ma.array(halpha_vel, mask = stellar_mask)
        
      #  set equal limits for color bar
        val_max = mhalpha_vel.max()
        val_min = mhalpha_vel.min()
        if (val_max >= abs(val_min)):
            lim = val_max
        else:
            lim = abs(val_min)
        
        if (lim > max_lim):
            lim = max_lim

        #create subplot for mhalpha_vel
        im = axes[2].imshow(mhalpha_vel, cmap ='bwr', vmin = -lim,vmax = lim)
        axes[2].set_xlabel('spaxel')
        axes[2].set_ylabel('spaxel')
        axes[2].set_title('Masked H-alpha Velocity')
        axes[2].invert_yaxis()

        fig.colorbar(im, ax = axes[2],label = 'Velocity [km/s]' )
        #plt.close()


        mstellar_vel = ma.array(stellar_vel, mask = stellar_mask)

        #create subplot for mhalpha_vel
        val_max = mstellar_vel.max()
        val_min = mstellar_vel.min()
        if (val_max >= abs(val_min)):
            lim = val_max
        else:
            lim = abs(val_min)

        if (lim > max_lim):
            lim = max_lim
        #create subplot for mstellar_vel
        im = axes[0].imshow(mstellar_vel, cmap ='bwr',vmin = -lim,vmax = lim)
        axes[0].set_xlabel('spaxel')
        axes[0].set_ylabel('spaxel')
        axes[0].set_title('Masked Stellar Velocity')
        axes[0].invert_yaxis()

        fig.colorbar(im, ax = axes[0],label = 'Velocity [km/s]' )
       # plt.close()

        #create subplot for flux

        im = axes[1].imshow(flux, cmap ='hot',vmin=0)
        axes[1].set_title('Flux')
        axes[1].set_xlabel('spaxel')
        axes[1].set_ylabel('spaxel')
        
        axes[1].invert_yaxis()
        
        fig.colorbar(im, ax = axes[1],label = '10-17 erg/s/cm2/Å/spaxel' )
       
        plt.savefig(data_folder + 'Plots/Subplots/'+ plateifu +'_subplot.png')
        plt.close(fig)
       
        
    except:
        with open(data_folder + 'Plots/ODD_subplot.txt', "a") as odds:  #writes plateifu's of bad rows in txt file
            odds.write(str(plateifu+'\n'))
        
        continue

In [ ]:
#ELPETRO_TH90_R
#add r90 column to table
#